# Apply to Other Datasets

## [Task 5] Create a Model of MNIST

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
# Download the MNIST dataset

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# Flatten

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

In [4]:
# Transform unit8 to float

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

In [5]:
# One hot encoder

# Initialize
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Fit
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])

# Transform
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [7]:
# Split the dataframe into train and test datasets

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)

In [8]:
class GetMiniBatch:
    """
    Iterator to get mini-batches
    
    Parameters
    ----------
    X : ndarray, shape (n_samples, n_features)
        Train data
    
    y : ndarray, shape (n_samples, 1)
        Correct values
    
    batch_size : int
        Size of a batch
    
    seed : int
        Seed of random values of NumPy
    """
    
    def __init__(self, X, y, batch_size=10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    
    
    def __len__(self):
        return self._stop
    
    
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        
        return self.X[p0:p1], self.y[p0:p1]
    
    
    def __iter__(self):
        self._counter = 0
        
        return self
    
    
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        
        self._counter += 1
        
        return self.X[p0:p1], self.y[p0:p1]

In [9]:
# Set hyperparameters

learning_rate = 0.01
batch_size = 10
num_epochs = 10

In [10]:
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

In [11]:
# Decide shapes of arguments given to a dataflow graph

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [12]:
# Mini-batch interator of train dataset

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [13]:
def example_net(x):
    """
    Simple neural network of 3 layers
    """
    
    # Declaration of weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    # 1st layer
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # 2nd layer
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # 3rd layer
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    
    return layer_output

In [14]:
# Read network construction

logits = example_net(X)

In [15]:
# Objective function

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

In [16]:
# Optimization method

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [17]:
# Prediction results

correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))

In [18]:
# Compute index values

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
# Initialize variables

init = tf.global_variables_initializer()

In [21]:
# Run the dataflow graph

with tf.Session() as sess:
    sess.run(init)
    
    # Loop per epoch
    for epoch in range(num_epochs):
        # Loop per mini-batch
        for mini_batch_x, mini_batch_y in get_mini_batch_train:
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
        
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 1.2742, val_loss : 0.7994, acc : 0.900, val_acc : 0.815
Epoch 1, loss : 0.2617, val_loss : 0.4815, acc : 0.900, val_acc : 0.879
Epoch 2, loss : 0.0402, val_loss : 0.3361, acc : 1.000, val_acc : 0.919
Epoch 3, loss : 0.1659, val_loss : 0.3004, acc : 0.900, val_acc : 0.924
Epoch 4, loss : 0.0062, val_loss : 0.2873, acc : 1.000, val_acc : 0.928
Epoch 5, loss : 0.2711, val_loss : 0.3083, acc : 0.900, val_acc : 0.925
Epoch 6, loss : 0.0656, val_loss : 0.2739, acc : 1.000, val_acc : 0.930
Epoch 7, loss : 0.0504, val_loss : 0.2720, acc : 1.000, val_acc : 0.934
Epoch 8, loss : 0.0322, val_loss : 0.3338, acc : 1.000, val_acc : 0.930
Epoch 9, loss : 0.2006, val_loss : 0.2752, acc : 0.900, val_acc : 0.932
test_acc : 0.937
